In [45]:
# # intall packages if you don't have
# !pip install tensorflow
# !pip install scikeras
# !pip install scikit-learn imbalanced-learn

In [46]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
# from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# Load Data

Upload the files to your Google Drive and mount it in the Colab notebook using the code below.

Files: `promos.csv.gz`, `test_history.csv.gz`, `train_history.csv.gz`, and `transactions.csv.gz`.

In [47]:
# This will ask you to give Colab permission to access your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# MODIFY THIS LINE with the path to where you saved the datafiles on your Google drive
path = '/content/drive/MyDrive/RSM8521 Deep Learning/A3'

In [49]:
promos = pd.read_csv(path + '/promos.csv.gz')
promos.head()

,promo,category,promoqty,manufacturer,promoval,brand
0,209524,51006740,1,1593002228,1.00,189092190
1,176321,42760698,1,1957688344,0.75,1386413202
2,17125,56844954,1,629075831,1.50,1429623935
3,177412,56844954,1,629075831,1.50,1429623935
4,69839,56844954,1,629075831,1.50,1429623935


In [50]:
train_history = pd.read_csv(path + '/train_history.csv.gz', parse_dates=['promodate'])
train_history.head()

,id,store,promo,region,promodate,active
0,115562959531,860548,176321,243820,2013-03-25,0.0
1,175261390705,625933,37568,191747,2013-03-15,0.0
2,273751574633,527828,209524,165933,2013-03-23,1.0
3,166923268906,1056730,214816,5506,2013-03-20,0.0
4,135410903443,241369,176321,278897,2013-03-25,0.0


In [51]:
test_history = pd.read_csv(path + '/test_history.csv.gz', parse_dates=['promodate'])
test_history.head()

,id,store,promo,region,promodate,active
0,250462324156,457755,96852,191747,2013-04-22,NaN
1,61214568335,740794,54956,203586,2013-04-23,NaN
2,274776763999,394092,95011,42773,2013-04-02,NaN
3,91546215019,876095,108237,323706,2013-04-24,NaN
4,220718580189,625933,222088,191747,2013-04-18,NaN


In [ ]:
# load productsize and amt columns as float32 to reduce memory footprint
transactions = pd.read_csv(path + '/transactions.csv.gz',
                           parse_dates=['date'],
                           dtype={'productsize': 'float32',
                                  'amt': 'float32'})
transactions.head()

In [ ]:
def check_existence(source_df, target_df, column_name):
    """
    Return a list of values from the source_df[column_name] that do not exist in target_df[column_name].
    """
    # Get unique values from both DataFrames
    source_unique_values = source_df[column_name].unique()
    target_unique_values = target_df[column_name].unique()

    # Check which source values are not in target
    not_in_target = ~np.isin(source_unique_values, target_unique_values)

    # Filter values that do not exist in target
    values_not_in_target = source_unique_values[not_in_target]

    return values_not_in_target

# Using the function to find stores from 'train_history', 'test_history' that are not in 'transactions'
stores_not_in_transactions_test = check_existence(test_history, transactions, 'store')
stores_not_in_transactions_train = check_existence(train_history, transactions, 'store')

# Display the result
print(f"Stores in test_history that are not in transactions: {stores_not_in_transactions_test}")
print(f"Stores in train_history that are not in transactions: {stores_not_in_transactions_train}")

In [ ]:
# Using the function to find stores from 'test_history' that are not in 'transactions'
id_not_in_transactions_test = check_existence(test_history, transactions, 'id')
id_not_in_transactions_train = check_existence(train_history, transactions, 'id')
# Display the result
print(f"Customers in test_history that are not in transactions: {id_not_in_transactions_test}")
print(f"Customers in train_history that are not in transactions: {id_not_in_transactions_train}")

In [ ]:
# Using the function to find stores from 'test_history' that are not in 'transactions'
region_not_in_train_test = check_existence(test_history, train_history, 'region')
# Display the result
print(f"region in test_history that are not in train_history: {region_not_in_train_test}")

# EDA

### transaction

In [ ]:
# Step 1: Group by customer and count the number of unique stores visited
customer_unique_stores = transactions.groupby('id')['store'].nunique().reset_index(name='unique_store_count')

# Step 2: Filter customers who visited multiple stores
customers_visiting_multiple_stores = customer_unique_stores[customer_unique_stores['unique_store_count'] > 1]

# Display customers who visited multiple stores
print(customers_visiting_multiple_stores)

In [ ]:
# Plot boxplot of 'amt' column
plt.figure(figsize=(10, 6))
transactions.boxplot(column='amt', vert=False, patch_artist=True, boxprops=dict(facecolor='skyblue'))
plt.xlabel('Amount')
plt.title('Boxplot of Transaction Amounts')
plt.show()

In [ ]:
# Check for negative amounts
negative_amounts = transactions[transactions['amt'] < 0]

# Display any rows with negative amounts
if not negative_amounts.empty:
    print("There are negative amounts in the 'amt' column:")
    # print(negative_amounts)
else:
    print("There are no negative amounts in the 'amt' column.")

In [ ]:
# Count the number of transactions per store
store_counts = transactions['store'].value_counts().head(15)

# Plotting
plt.figure(figsize=(10, 6))
store_counts.plot(kind='bar')
plt.title('Distribution of Transactions by Store')
plt.xlabel('Store ID')
plt.ylabel('Number of Transactions')
plt.show()

In [ ]:
# Converting 'date' to datetime type if not already
transactions['date'] = pd.to_datetime(transactions['date'])

# Summing amount over each date
time_series = transactions.groupby('date')['amt'].sum()

# Plotting
plt.figure(figsize=(10, 6))
time_series.plot()
plt.title('Sales Amount Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales Amount')
plt.show()

In [ ]:
# Extracting the weekday from the date (Monday=0, Sunday=6)
transactions['weekday'] = transactions['date'].dt.weekday

# Mapping weekday numbers to names for better visualization
weekday_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
transactions['weekday_name'] = transactions['weekday'].map(weekday_names)

# Summing amount over each weekday
weekday_sales = transactions.groupby('weekday_name')['amt'].sum().sort_values(ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
weekday_sales.plot(kind='bar')
plt.title('Sales Amount by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Total Sales Amount')
plt.xticks(rotation=45)  # Rotate labels for better readability
plt.show()

In [ ]:
# Extracting the month from the date
transactions['month'] = transactions['date'].dt.month

# Mapping month numbers to names for better visualization
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April',
    5: 'May', 6: 'June', 7: 'July', 8: 'August',
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
}
transactions['month_name'] = transactions['month'].map(month_names)

# Summing amount over each month
monthly_sales = transactions.groupby('month_name')['amt'].sum().sort_values(ascending=False)

# Plotting
plt.figure(figsize=(12, 7))
monthly_sales.plot(kind='bar')
plt.title('Sales Amount by Month')
plt.xlabel('Month')
plt.ylabel('Total Sales Amount')
plt.xticks(rotation=45)  # Rotate labels for better readability
plt.show()

In [ ]:
print("there are 116 unique stores")
# Group by 'store' and count unique combinations of 'category', 'manufacturer', and 'brand' for each store
unique_products_per_store = transactions.groupby('store').apply(lambda df: df.drop_duplicates(subset=['category', 'manufacturer', 'brand']).shape[0])

# Display the result
unique_products_per_store

### promos

In [ ]:
# # Sum of promo values by category
# category_promo_values = promos.groupby('category')['promoval'].sum()

# # Plotting
# plt.figure(figsize=(10, 6))
# category_promo_values.plot(kind='bar')
# plt.title('Total Promotion Value by Category')
# plt.xlabel('Category')
# plt.ylabel('Total Promotional Value')
# plt.show()

In [ ]:
# # Count the number of promotions by manufacturer
# manufacturer_promos = promos['manufacturer'].value_counts()

# # Plotting
# plt.figure(figsize=(10, 6))
# manufacturer_promos.head(10).plot(kind='bar')  # Showing top 10 only for brevity
# plt.title('Count of Promotions by Manufacturer (Top 10)')
# plt.xlabel('Manufacturer')
# plt.ylabel('Number of Promotions')
# plt.show()

In [ ]:
# # Average promotion value by brand
# brand_promo_average = promos.groupby('brand')['promoval'].mean()

# # Plotting
# plt.figure(figsize=(10, 6))
# brand_promo_average.sort_values(ascending=False).head(10).plot(kind='bar')  # Top 10 brands by average promo value
# plt.title('Average Promotion Value by Brand (Top 10)')
# plt.xlabel('Brand')
# plt.ylabel('Average Promotional Value')
# plt.show()

# Extract Features

## Compute RFM Features

In [ ]:
max_date = transactions.date.max()
transactions['last_purchase'] = (max_date - transactions['date']) / np.timedelta64(1, "D")

# Agg transaction data to id
trans_features = transactions.groupby('id').agg({
    'last_purchase': 'min',
    'date': ['nunique', 'min'],
    'amt': 'sum',
})

# Compute RFM columns
trans_features['recency'] = trans_features['last_purchase']['min']
trans_features['frequency'] = (trans_features['date']['nunique']
                               / ((max_date - trans_features['date']['min']) / np.timedelta64(1, "D")))
trans_features['monetary'] = trans_features['amt']['sum']

# Select out required features
trans_features = trans_features[['recency', 'frequency', 'monetary']].reset_index()
trans_features.columns = trans_features.columns.get_level_values(0)
trans_features.head()

## Convert Datetime

In [ ]:
# Rank the sorted sales, with 1 being the highest sales
weekday_ranks = weekday_sales.rank(method='min', ascending=False)
# Convert the ranks to a dictionary
weekday_rank_dict = weekday_ranks.to_dict()
weekday_rank_dict

In [ ]:
# Rank the sorted sales, with 1 being the highest sales
month_ranks = monthly_sales.rank(method='min', ascending=False)
# Convert the ranks to a dictionary
month_rank_dict = month_ranks.to_dict()
month_rank_dict

## Encode Catagorical Features in transactions

In [ ]:
unique_manufacturers = len(transactions['manufacturer'].unique())
unique_categories = len(transactions['category'].unique())
unique_brands = len(transactions['brand'].unique())

print(f"Unique manufacturers: {unique_manufacturers}")
print(f"Unique categories: {unique_categories}")
print(f"Unique brands: {unique_brands}")

In [ ]:
# Calculate popularity counts for 'category'
category_counts = transactions['category'].value_counts()
# Convert to dictionary
category_popularity_dict = category_counts.to_dict()

# Calculate popularity counts for 'manufacturer'
manufacturer_counts = transactions['manufacturer'].value_counts()
# Convert to dictionary
manufacturer_popularity_dict = manufacturer_counts.to_dict()

# Calculate popularity counts for 'brand'
brand_counts = transactions['brand'].value_counts()
# Convert to dictionary
brand_popularity_dict = brand_counts.to_dict()

# Output the dictionaries to verify
print("Category Popularity Dict:", category_popularity_dict)
print("Manufacturer Popularity Dict:", manufacturer_popularity_dict)
print("Brand Popularity Dict:", brand_popularity_dict)

This approach groups  entities into 10 popularity levels, with 1 being the least popular and 10 the most popular, based on their frequency in the dataset.

## Store info

In [ ]:
# Define store size based on number of unique products in a store.
# product number is defined by grouping by 'store' and counting unique combinations of 'category', 'manufacturer', and 'brand' for each store
store_size_dict = unique_products_per_store.to_dict()
print(store_size_dict)

In [ ]:
# store_sales = transactions.groupby('store')['amt'].sum().reset_index()
# store_sales.rename(columns={'amt': 'store_sales'}, inplace=True)
# store_sales_dict = dict(zip(store_sales['store'], store_sales['store_sales']))
# print(store_sales_dict)

In [ ]:
# # Group transactions by store and calculate the number of unique product sizes
# unique_product_sizes_per_store = transactions.groupby('store')['productsize'].nunique().reset_index()
# unique_product_sizes_per_store_dict = dict(zip(unique_product_sizes_per_store['store'], unique_product_sizes_per_store['productsize']))
# print(unique_product_sizes_per_store_dict)

## Customers

A higher ratio might indicate less price sensitivity (returns are more significant when they happen)

In [ ]:
# # Filter out the return transactions (assuming returns are indicated by negative amounts)
# returns = transactions[transactions['amt'] < 0]

# # Aggregate the total return amount by customer
# return_history = returns.groupby('id')['amt'].sum().reset_index(name='total_return_amount')

# # Count the number of return instances by customer and convert to DataFrame
# return_times = returns.groupby('id').size().reset_index(name='total_return_time')

# # Merge the total return amount and return times DataFrames on 'id'
# return_history = return_history.merge(return_times, on='id')

# # Calculate return behavior
# return_history['return_behavior'] = return_history['total_return_amount'].abs() / return_history['total_return_time']

# # Replace infinite values with NaN if any division by zero occurred, then fill with a sensible value
# return_history['return_behavior'].replace([np.inf, -np.inf], np.nan, inplace=True)
# return_history['return_behavior'].fillna(0, inplace=True)  # Assuming 0 for when there's no returns

# # Convert the return history to a dictionary
# return_history_abs_dict = return_history.set_index('id')['return_behavior'].to_dict()

# # Output the dictionary to verify
# print(return_history_abs_dict)

In [ ]:
# transactions['date'] = pd.to_datetime(transactions['date'])
# transactions_2012 = transactions[(transactions['date'].dt.year == 2012) &
#                                  (transactions['date'].dt.month.isin([3, 4]))]
# customer_spending = transactions_2012.groupby(['id', transactions_2012['date'].dt.month])['amt'].sum().reset_index()
# customer_spending.rename(columns={'date': 'month', 'amt': 'total_spent'}, inplace=True)
# march_spending = customer_spending[customer_spending['month'] == 3]
# april_spending = customer_spending[customer_spending['month'] == 4]

# march_dict = dict(zip(march_spending['id'], march_spending['total_spent']))
# april_dict = dict(zip(april_spending['id'], april_spending['total_spent']))

# # Output the dictionaries to verify
# print("March spending:", march_dict)
# print("April spending:", april_dict)

In [ ]:
# Calculate the standard deviation of 'amt' for each customer
std_transaction_customer = transactions.groupby('id')['amt'].std()

# Convert the Series to a dictionary
std_transaction_customer_dict = std_transaction_customer.to_dict()

# Display the dictionary
print(std_transaction_customer_dict)

In [ ]:
# # Step 1: Aggregate the total amount spent by each customer
# customer_spending = transactions.groupby('id')['amt'].sum().reset_index()
# customer_spending.rename(columns={'amt': 'total_spent'}, inplace=True)

# # Step 2: Rank customers into deciles based on total spent
# customer_spending['customer_id_rank'] = pd.qcut(customer_spending['total_spent'], 20, labels=False) + 1

# id_rank_dict = customer_spending.set_index('id')['customer_id_rank'].to_dict()
# # Output the dictionary to verify
# print(id_rank_dict)

In [ ]:
# # Group transactions by customer and store, then count the number of transactions
# customer_store_visits = transactions.groupby(['id', 'store']).size().reset_index(name='visit_count')
# customer_store_visits_dict = customer_store_visits.set_index('id')['visit_count'].to_dict()
# # Display the DataFrame with visit count for each customer-store combination
# print(customer_store_visits_dict)

In [ ]:
# Group by customer ID and manufacturer, then count occurrences
manufacturer_counts_per_customer = transactions.groupby(['id', 'manufacturer']).size().reset_index(name='count')

# Calculate the total number of transactions per customer
total_transactions_per_customer = transactions.groupby('id').size().reset_index(name='total')

# Merge the counts with total transactions to calculate percentages
merged_data = pd.merge(manufacturer_counts_per_customer, total_transactions_per_customer, on='id')

# Calculate percentage of total transactions for each manufacturer per customer
merged_data['percentage'] = (merged_data['count'] / merged_data['total']) * 100

# Sort the results by customer ID and the percentage in descending order to see most significant first
sorted_manufacturers_per_customer = merged_data.sort_values(by=['id', 'percentage'], ascending=[True, False])

# Create a dictionary with key as (id, manufacturer) and value as percentage
customer_manufacturer_dict = dict(zip(zip(merged_data['id'], merged_data['manufacturer']), merged_data['percentage']))

# # Output the dictionary to verify
# customer_manufacturer_dict

In [ ]:
# Group by customer ID and brand, then count occurrences
brand_counts_per_customer = transactions.groupby(['id', 'brand']).size().reset_index(name='count')

# Calculate the total number of transactions per customer
total_transactions_per_customer = transactions.groupby('id').size().reset_index(name='total')

# Merge the counts with total transactions to calculate percentages
merged_data_brand = pd.merge(brand_counts_per_customer, total_transactions_per_customer, on='id')

# Calculate percentage of total transactions for each brand per customer
merged_data_brand['percentage'] = (merged_data_brand['count'] / merged_data_brand['total']) * 100

# Sort the results by customer ID and the percentage in descending order
sorted_brands_per_customer = merged_data_brand.sort_values(by=['id', 'percentage'], ascending=[True, False])

# Create a dictionary with key as (id, brand) and value as percentage
customer_brand_dict = dict(zip(zip(merged_data_brand['id'], merged_data_brand['brand']), merged_data_brand['percentage']))

# # Output the dictionary to verify
# print(customer_brand_dict)

In [ ]:
# Group by customer ID and category, then count occurrences
category_counts_per_customer = transactions.groupby(['id', 'category']).size().reset_index(name='count')

# Calculate the total number of transactions per customer
total_transactions_per_customer = transactions.groupby('id').size().reset_index(name='total')

# Merge the counts with total transactions to calculate percentages
merged_data_category = pd.merge(category_counts_per_customer, total_transactions_per_customer, on='id')

# Calculate percentage of total transactions for each category per customer
merged_data_category['percentage'] = (merged_data_category['count'] / merged_data_category['total']) * 100

# Sort the results by customer ID and the percentage in descending order
sorted_categories_per_customer = merged_data_category.sort_values(by=['id', 'percentage'], ascending=[True, False])

# Create a dictionary with key as (id, brand) and value as percentage
customer_category_dict = dict(zip(zip(merged_data_category['id'], merged_data_category['category']), merged_data_category['percentage']))

# # Output the dictionary to verify
# print(customer_category_dict)

In [ ]:
# # Unique Categories Purchased
# customer_category = transactions.groupby('id')['category'].nunique().reset_index(name='category_favor')
# customer_category_num_dict = customer_category.set_index('id')['category_favor'].to_dict()
# print(customer_category_num_dict)
# # Unique Brand Purchased
# customer_brand = transactions.groupby('id')['brand'].nunique().reset_index(name='brand_favor')
# customer_brand_num_dict = customer_brand.set_index('id')['brand_favor'].to_dict()
# print(customer_brand_num_dict)
# # Unique manufacturer Purchased
# customer_manufacturer = transactions.groupby('id')['manufacturer'].nunique().reset_index(name='manufacturer_favor')
# customer_manufacturer_num_dict = customer_manufacturer.set_index('id')['manufacturer_favor'].to_dict()
# print(customer_manufacturer_num_dict)

# Generate train sets

In [ ]:
train_history = pd.read_csv(path + '/train_history.csv.gz', parse_dates=['promodate'])

### modify train_history

In [ ]:
# Convert 'promodate' to datetime
train_history['promodate'] = pd.to_datetime(train_history['promodate'])

# # Extract month and weekday from 'promodate'
# # train_history['month'] = train_history['promodate'].dt.month  # Extracts the month as a number
train_history['weekday'] = train_history['promodate'].dt.weekday  # Extracts the weekday as a number (Monday=0, Sunday=6)

# # Optionally, you might want to have the month and weekday names instead of numbers
# # train_history['month_name'] = train_history['promodate'].dt.strftime('%B')  # Extracts the name of the month
# train_history['weekday_name'] = train_history['promodate'].dt.strftime('%A')  # Extracts the name of the weekday

# # Map the month and weekday names to their ranks
# # train_history['month_rank'] = train_history['month_name'].map(month_rank_dict)
# train_history['weekday_rank'] = train_history['weekday_name'].map(weekday_rank_dict)
# train_history['weekday'] = train_history['weekday']+1
# train_history.drop(columns=['promodate', 'weekday_name'], inplace=True)

# train_history['region_popularity'] = train_history['region'].map(region_popularity_dict)
# # Display the DataFrame to verify the changes
# train_history.head()

In [ ]:
train_history = train_history.merge(promos, on='promo', how='left')

# Map the category, manufacturer and brand to their ranks
train_history['category_rank'] = train_history['category'].map(category_popularity_dict)
train_history['manufacturer_rank'] = train_history['manufacturer'].map(manufacturer_popularity_dict)
# train_history['brand_rank'] = train_history['brand'].map(brand_popularity_dict)
# train_history['total_rank'] = train_history['category_rank'] + train_history['manufacturer_rank'] + train_history['brand_rank']

# Map the store factors
train_history['store_size'] = train_history['store'].map(store_size_dict)
# train_history['store_sale'] = train_history['store'].map(store_sales_dict)
# train_history['store_performance'] = train_history['store_sale']/train_history['store_size']

# Map the customer factors
# General function to retrieve customer preferences from a dictionary
def get_customer_preference(row, dictionary, attribute_name):
    return dictionary.get((row['id'], row[attribute_name]), 0)

train_history['customer_category_favor'] = train_history.apply(lambda row: get_customer_preference(row, customer_category_dict, 'category'), axis=1)
train_history['customer_brand_favor'] = train_history.apply(lambda row: get_customer_preference(row, customer_brand_dict, 'brand'), axis=1)
train_history['customer_manufacturer_favor'] = train_history.apply(lambda row: get_customer_preference(row, customer_manufacturer_dict, 'manufacturer'), axis=1)

# train_history['customer_store_visits'] = train_history['id'].map(customer_store_visits_dict)
# train_history.customer_store_visits.fillna(0, inplace=True)
# train_history['std_transaction_customer'] = train_history['id'].map(std_transaction_customer_dict)
# train_history['std_transaction_customer'].fillna(0, inplace=True)
# train_history['customer_rank'] = train_history['id'].map(id_rank_dict)
# train_history['customer_rank'].fillna(0, inplace=True)
# train_history['return_behavior'] = train_history['id'].map(return_history_abs_dict)
# train_history['return_behavior'].fillna(0, inplace=True)
# train_history['customer_total_category'] = train_history['id'].map(customer_category_num_dict)
# train_history['customer_total_brand'] = train_history['id'].map(customer_brand_num_dict)
# train_history['customer_total_manufacturer'] = train_history['id'].map(customer_manufacturer_num_dict)
# train_history['customer_total_category'].fillna(0, inplace=True)
# train_history['customer_total_brand'].fillna(0, inplace=True)
# train_history['customer_total_manufacturer'].fillna(0, inplace=True)
# Display the DataFrame to verify the changes
train_history.head()

In [ ]:
# Initialize the LabelEncoder
label_encoder_region = LabelEncoder()

# Fit the label encoder and transform the 'region' column
train_history['region_encoded'] = label_encoder_region.fit_transform(train_history['region'])

In [ ]:
# Generate train/test features from trans_features
train_history = train_history.merge(trans_features, on='id', how='left')
train_history.recency.fillna(365, inplace=True)
train_history.frequency.fillna(0, inplace=True)
train_history.monetary.fillna(0, inplace=True)
train_history['recency'] = train_history['recency'] + train_history['promodate'].dt.day
# Display the DataFrame to verify the changes
train_history.drop(columns=['category', 'manufacturer', 'brand', 'store','promoqty'], inplace=True)
train_history.head()

In [ ]:
train_history.active.value_counts()

In [ ]:
train_history.columns.values

In [ ]:
# # 'month', 'promoqty', 'month_rank'
# x_cols = ['promoval',
#        'total_rank', 'store_performance',
#        'customer_category_favor', 'customer_brand_favor',
#        'customer_manufacturer_favor', 'std_transaction_customer',
#        'recency', 'frequency', 'monetary']
x_cols = ['weekday',
       'promoval', 'category_rank', 'manufacturer_rank', 'store_size',
       'customer_category_favor', 'customer_brand_favor',
       'customer_manufacturer_favor','region_encoded',
       'recency', 'frequency', 'monetary']
y_col = 'active'

In [ ]:
columns = x_cols + [y_col]  # Append the target column to the list of feature columns

# Calculate the correlation matrix
correlation = train_history[columns].corr()

# Plotting the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

### not useful

In [ ]:
# unique_manufacturers = len(train_history['manufacturer'].unique())
# unique_categories = len(train_history['category'].unique())
# unique_brands = len(train_history['brand'].unique())

# print(f"Unique manufacturers: {unique_manufacturers}")
# print(f"Unique categories: {unique_categories}")
# print(f"Unique brands: {unique_brands}")

In [ ]:
# # Convert date columns to datetime if not already
# transactions['date'] = pd.to_datetime(transactions['date'])
# train_history['promodate'] = pd.to_datetime(train_history['promodate'])

# # Filter transactions to only include those from 2012
# transactions_2012 = transactions[transactions['date'].dt.year == 2012]

# # Sum the transaction amounts for each date in 2012
# daily_sums_2012 = transactions_2012.groupby(transactions_2012['date'].dt.date)['amt'].sum()

# # Function to adjust the year of promodate to 2012
# def adjust_year_to_2012(date):
#     return date.replace(year=2012)

# # Apply this function to promodate to create a new column for adjusted dates
# train_history['adjusted_date'] = train_history['promodate'].apply(adjust_year_to_2012)

# # Map the summed amounts from 2012 to the adjusted dates in train_history
# train_history['same_day_amt_2012'] = train_history['adjusted_date'].map(daily_sums_2012)

# # Drop the adjusted_date column if no longer needed
# train_history.drop(columns=['adjusted_date'], inplace=True)
# train_history['same_day_amt_2012'].fillna(0, inplace=True)
# # # Output the train_history to verify
# # train_history.head()

In [ ]:
# # Convert date columns to datetime if not already
# transactions['date'] = pd.to_datetime(transactions['date'])
# train_history['promodate'] = pd.to_datetime(train_history['promodate'])

# # Create aggregates before merging
# # # Aggregate by manufacturer
# # agg_manufacturer = transactions.groupby(['id', 'manufacturer']).agg(previous_spend_manufacturer=('amt', 'sum')).reset_index()

# # # Aggregate by brand
# # agg_brand = transactions.groupby(['id', 'brand']).agg(previous_spend_brand=('amt', 'sum')).reset_index()

# # Aggregate by category
# agg_category = transactions.groupby(['id', 'category']).agg(previous_purchases_time_category=('id', 'count')).reset_index()

# # Aggregate by all - count the number of transactions instead of summing the amount
# agg_all = transactions.groupby(['id', 'manufacturer', 'brand', 'category']).agg(previous_purchases_time_exact=('id', 'count')).reset_index()

# # Merge each aggregate back into train_history on respective keys
# # train_history = train_history.merge(agg_manufacturer, on=['id', 'manufacturer'], how='left')
# # train_history = train_history.merge(agg_brand, on=['id', 'brand'], how='left')
# train_history = train_history.merge(agg_category, on=['id', 'category'], how='left')
# train_history = train_history.merge(agg_all, on=['id', 'manufacturer', 'brand', 'category'], how='left')

# # Fill NaN values with 0 if no transactions matched
# # train_history.fillna({'previous_spend_manufacturer': 0, 'previous_spend_brand': 0, 'previous_spend_category': 0, 'previous_spend_exact':0}, inplace=True)

# train_history.fillna({'previous_purchases_time_category':0, 'previous_purchases_time_exact':0}, inplace=True)

# # train_history.head()

# Build Model

In [ ]:
#@title helper function - backward_feature_elimination
# from sklearn.model_selection import cross_val_score
# def backward_feature_elimination(X, y, initial_features, estimator, scoring='roc_auc', cv=5):
#     """
#     Perform backward feature elimination using an XGBoost model.

#     Parameters:
#     X (DataFrame): Feature dataset
#     y (Series): Target variable
#     initial_features (list): List of initial features to consider
#     estimator (XGBClassifier or XGBRegressor): The XGBoost model
#     scoring (str): Scoring metric name for model evaluation
#     cv (int): Number of cross-validation folds

#     Returns:
#     list: List of selected features
#     """
#     features = initial_features.copy()
#     best_score = np.mean(cross_val_score(estimator, X[initial_features], y, scoring=scoring, cv=cv))
#     selected_features = features.copy()

#     while len(features) > 0:
#         scores = {}
#         for feature in features:
#             temp_features = [f for f in features if f != feature]
#             score = np.mean(cross_val_score(estimator, X[temp_features], y, scoring=scoring, cv=cv))
#             scores[feature] = score

#         best_feature = max(scores, key=scores.get)
#         if scores[best_feature] > best_score:
#             best_score = scores[best_feature]
#             selected_features = [f for f in features if f != best_feature]
#             print(f"Removing feature {best_feature} improved score to {best_score}. Remaining features: {len(selected_features)}")
#             features.remove(best_feature)
#         else:
#             print(f"No improvement by removing any feature. Stopping.")
#             break

#     return selected_features

### split dataset

In [ ]:
X = train_history[x_cols]
y = train_history[y_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# # Apply SMOTE
# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

# # Rescale the features
# scaler = StandardScaler().fit(X_train_resampled)  # Fit scaler to the oversampled training set
# X_train_scaled = scaler.transform(X_train_resampled)
# X_test_scaled = scaler.transform(X_test)  # Apply the same scaler to the test set

# # Rescale the features
# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

## Xgboost

In [ ]:
import xgboost as xgb

# Initialize XGBoost model
xg_model = xgb.XGBClassifier(objective='binary:logistic')

# Fit the model
xg_model.fit(X_train, y_train)
y_pred = xg_model.predict(X_test)

print(classification_report(y_test, y_pred))
print('Test Accuracy: {:.5f}'.format(xg_model.score(X_test, y_test)))
print('Test AUC: {:.5f}'.format(roc_auc_score(y_test, xg_model.predict_proba(X_test)[:, 1])))

In [ ]:
# # Initialize the XGBoost classifier
# model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.03, max_depth=4, min_child_weight=7, gamma=0.3, colsample_bytree = 1.0, subsample = 0.5)

# @title backward feature selection
# selected_features = backward_feature_elimination(X_train, y_train, x_cols, model)
# print("Selected features:", selected_features)

# selected_features = ['promoval', 'total_rank', 'store_performance', 'customer_category_favor', 'customer_brand_favor', 'customer_rank', 'std_transaction_customer', 'return_behavior', 'recency']

# X = train_history[selected_features]
# y = train_history[y_col]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Model performance decreased after backward feature selection

 precision    recall  f1-score   support

         0.0       0.83      0.96      0.89      3201
         1.0       0.52      0.19      0.28       799

    accuracy                           0.80      4000
   macro avg       0.67      0.57      0.58      4000
weighted avg       0.76      0.80      0.76      4000

Test Accuracy: 0.80300
Test AUC: 0.66471

Model performance decreased after feature selection

In [ ]:
# # from sklearn.model_selection import RandomizedSearchCV
# #@title hyperparameter tuning
# param_grid = {
#     'max_depth': [3, 4, 5, 6],  # Slightly increasing max_depth to try capturing more complex patterns
#     'n_estimators': [100, 150, 200],  # Evaluating a modest increase in the number of trees
#     'learning_rate': [0.01, 0.03, 0.05, 0.1],  # Experimenting around the current learning rate
#     # 'min_child_weight': [2, 3, 4, 5, 6, 7],  # Considering even lower values for sensitivity to the minority class
#     # 'gamma': [0, 0.1, 0.3],  # Gamma can help with regularization and prevent overfitting
#     # 'colsample_bytree': [0.5, 0.7, 1.0],  # Balancing the number of features in each tree
#     # 'subsample': [0.5, 0.7, 1.0],  # Balancing the number of samples in each tree
#     'scale_pos_weight': [1, 1.5, 2, 2.5, (3201 / 799)]  # Balancing the minority class
# }

# # Create a base model
# model = xgb.XGBClassifier(objective='binary:logistic')

# # Setup GridSearchCV
# grid_search = GridSearchCV(
#     estimator=model,
#     param_grid=param_grid,
#     cv=5,
#     scoring='roc_auc',  # Optimize for ROC AUC
#     n_jobs=1  # Use all available CPU cores
# )

# # Fit GridSearchCV to the training data
# grid_search_result = grid_search.fit(X_train, y_train)

# # Output the best parameters and best score
# print("Best AUC: %f using %s" % (grid_search_result.best_score_, grid_search_result.best_params_))

Best AUC: 0.690430 using {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'scale_pos_weight': 2.5}


In [ ]:
# Output the best parameters and best score
xg_model_grid = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, max_depth=3, scale_pos_weight=2.5)

# Fit the model
xg_model_grid.fit(X_train, y_train)
y_pred = xg_model_grid.predict(X_test)

print(classification_report(y_test, y_pred))
print('Test Accuracy: {:.5f}'.format(xg_model_grid.score(X_test, y_test)))
print('Test AUC: {:.5f}'.format(roc_auc_score(y_test, xg_model_grid.predict_proba(X_test)[:, 1])))

In [ ]:
xgb.plot_importance(xg_model_grid)  # Shows the top features
plt.title('Feature Importance')
plt.show()

# Best Model

In [ ]:
# Output the best parameters and best score
xg_model_best = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, max_depth=3, scale_pos_weight=2.5)

# Fit the model
xg_model_best.fit(X, y)
print('Train Accuracy: {:.5f}'.format(xg_model_best.score(X, y)))
print('Train AUC: {:.5f}'.format(roc_auc_score(y, xg_model_best.predict_proba(X)[:, 1])))

# Generate Prediction

In [ ]:
test_history = pd.read_csv(path + '/test_history.csv.gz', parse_dates=['promodate'])

### modify test_history

In [ ]:
# Fit the label encoder and transform the 'region' column
test_history['region_encoded'] = label_encoder_region.transform(test_history['region'])
# Convert 'promodate' to datetime
test_history['promodate'] = pd.to_datetime(test_history['promodate'])
# Extract month and weekday from 'promodate'
test_history['weekday'] = test_history['promodate'].dt.weekday  # Extracts the weekday as a number (Monday=0, Sunday=6)

In [ ]:
# Merge test_history with promos DataFrame
test_history = test_history.merge(promos, on='promo', how='left')

# Map the category, manufacturer, and brand to their ranks
test_history['category_rank'] = test_history['category'].map(category_popularity_dict)
test_history['manufacturer_rank'] = test_history['manufacturer'].map(manufacturer_popularity_dict)
test_history['brand_rank'] = test_history['brand'].map(brand_popularity_dict)
test_history['total_rank'] = test_history['category_rank'] + test_history['manufacturer_rank'] + test_history['brand_rank']

# Map the store factors
test_history['store_size'] = test_history['store'].map(store_size_dict)
# test_history['store_sale'] = test_history['store'].map(store_sales_dict)
# test_history['store_performance'] = test_history['store_sale'] / test_history['store_size']

# Map the customer factors using the generalized function
def get_customer_preference(row, dictionary, attribute_name):
    return dictionary.get((row['id'], row[attribute_name]), 0)

test_history['customer_category_favor'] = test_history.apply(lambda row: get_customer_preference(row, customer_category_dict, 'category'), axis=1)
test_history['customer_brand_favor'] = test_history.apply(lambda row: get_customer_preference(row, customer_brand_dict, 'brand'), axis=1)
test_history['customer_manufacturer_favor'] = test_history.apply(lambda row: get_customer_preference(row, customer_manufacturer_dict, 'manufacturer'), axis=1)

In [ ]:
# Merge test_history with the transaction features on 'id' and perform a left join
test_history = test_history.merge(trans_features, on='id', how='left')

# Fill missing values for 'recency', 'frequency', and 'monetary' with appropriate defaults
test_history.recency.fillna(365, inplace=True)  # Assuming no transaction within a year
test_history.frequency.fillna(0, inplace=True)  # Assuming no repeat transactions
test_history.monetary.fillna(0, inplace=True)   # Assuming no monetary transactions
test_history['recency'] = test_history['recency'] + test_history['promodate'].dt.day
# Display the DataFrame to verify the changes
test_history.drop(columns=['category', 'manufacturer', 'brand', 'store','promoqty'], inplace=True)
test_history.head()

final features: ['weekday',
       'promoval', 'category_rank', 'manufacturer_rank', 'store_size',
       'customer_category_favor', 'customer_brand_favor',
       'customer_manufacturer_favor', 'customer_rank', 'recency',
       'frequency', 'monetary']

In [ ]:
X_predict = test_history[x_cols]
# X_predict = test_history[selected_features]
predict_out = test_history[['id', 'active']].copy()
X_predict.head(10)

### predict

In [ ]:
predict_out['active'] = xg_model_best.predict_proba(X_predict)[:, 1]
# predict_out['active'] = best_forest.predict_proba(X_predict)[:, 1]
predict_out

In [ ]:
predict_out.to_csv('predict.csv', header=True, index=False)
predict_out.head()

In [ ]:
# This will download your prediction files
from google.colab import files
files.download('predict.csv')

# Not considered models



## RandomForestClassifier

In [ ]:
# model = RandomForestClassifier()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

# print(classification_report(y_test, y_pred))
# print('Test Accuracy: {:.5f}'.format(model.score(X_test, y_test)))
# print('Test AUC: {:.5f}'.format(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])))

In [ ]:
# # Expanded and enriched parameter grid
# param_grid_rfc = {
#     'n_estimators': [100, 200, 300, 500],  # Added more options for number of trees
#     'max_features': ['sqrt', 'log2', None],  # Added float options specifying the fraction of features to consider
#     'max_depth': [6, 9, 12, 15, None],  # Included None to allow unlimited depth
#     'max_leaf_nodes': [6, 9, 12, 15, None],  # Included None to allow unlimited leaf nodes
#     'min_samples_split': [2, 5, 10],  # Added minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Added minimum number of samples required to be at a leaf node
#     'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
# }

# # Create the RandomForest classifier base model
# rf_model = RandomForestClassifier()

# # Setup GridSearchCV
# grid_search = GridSearchCV(
#     estimator=rf_model,
#     param_grid=param_grid_rfc,
#     cv=5,
#     scoring='roc_auc',  # Use ROC AUC as the score to optimize
#     n_jobs=-1  # Using all available cores
# )

# # Fit GridSearchCV to the training data
# grid_search_result = grid_search.fit(X_train, y_train)

# # Output the best parameters and best score found by GridSearchCV
# print("Best estimator:", grid_search_result.best_estimator_)
# print("Best score:", grid_search_result.best_score_)


In [ ]:
# model_grid = RandomForestClassifier(bootstrap=False, max_depth=6, max_features='log2',
#                        min_samples_leaf=2, min_samples_split=10,
#                        n_estimators=500)
# model_grid.fit(X_train, y_train)
# y_pred_grid = model_grid.predict(X_test)
# print(classification_report(y_test, y_pred_grid))
# print('Test Accuracy: {:.5f}'.format(model_grid.score(X_test, y_test)))
# print('Test AUC: {:.5f}'.format(roc_auc_score(y_test, model_grid.predict_proba(X_test)[:, 1])))

In [ ]:
# # 'month', 'promoqty', 'month_rank', 'return_behavior','frequency'
# x_cols = [
#  'promoval',
#  'total_rank',
#  'store_performance',
#  'customer_category_favor',
#  'customer_brand_favor',
#  'customer_manufacturer_favor',
#  'std_transaction_customer',
#  'previous_purchases_time_exact',
#  'monetary']
# y_col = 'active'

# X = train_history[x_cols]
# y = train_history[y_col]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# # Get feature importances
# feature_importances = model_grid.feature_importances_

# # Create a DataFrame to display feature importances
# feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
# feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# # Plot feature importances
# plt.figure(figsize=(10, 6))
# plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='skyblue')
# plt.xlabel('Importance Score')
# plt.ylabel('Feature')
# plt.title('Feature Importance')
# plt.gca().invert_yaxis()  # Invert y-axis to show most important features at the top
# plt.show()

## ANN

In [ ]:
# from sklearn.utils import shuffle
# from sklearn import preprocessing
# from keras.layers import Dense, Dropout
# from keras.models import Sequential
# from keras.optimizers import Adam
# from keras.metrics import AUC
# from scikeras.wrappers import KerasClassifier

In [ ]:
# # create ANN model
# model = Sequential()

# # Define the 1st hidden layer
# model.add(Dense(70, input_dim = X_train_scaled.shape[1], activation='relu'))

# # model.add(Dropout(0.2))

# # Define the Output layer
# model.add(Dense(1, activation='sigmoid'))

# # Compiling the model
# model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy", AUC()])

# # Fitting the ANN to the Training set
# model.fit(X_train_scaled, y_train ,batch_size = 8, epochs = 70, verbose=1)

In [ ]:
# from sklearn.metrics import roc_auc_score, roc_curve
# # Make predictions with the model
# probabilities = model.predict(X_test_scaled)  # This will give you the probabilities of class 1
# # Calculate the ROC AUC score
# auc = roc_auc_score(y_test, probabilities)
# print(f"ROC AUC Score: {auc}")

In [ ]:
# # Calculate ROC curve
# fpr, tpr, thresholds = roc_curve(y_test, probabilities)

# # Create plot
# plt.figure(figsize=(8, 6))
# plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {auc:.2f})')
# plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
# plt.legend(loc="lower right")
# plt.show()

## LogisticRegression

In [ ]:
# from sklearn.linear_model import LogisticRegression
# # Model Training
# model_logit = LogisticRegression()
# model_logit.fit(X_train_scaled, y_train)

# # Model Evaluation
# print('Test Accuracy: {:.5f}'.format(model_logit.score(X_test_scaled, y_test)))
# print('Test AUC: {:.5f}'.format(roc_auc_score(y_test, model_logit.predict_proba(X_test_scaled)[:, 1])))

In [ ]:
# # Define the parameter grid to search
# param_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
#     'penalty': ['l1', 'l2'],  # Penalty term
#     'solver': ['liblinear', 'saga']  # Algorithm to use in optimization problem
# }

# # Create a logistic regression model
# model_logit = LogisticRegression()

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=model_logit, param_grid=param_grid, cv=5, scoring='roc_auc')
# grid_search.fit(X_train_scaled, y_train)

# # Get the best hyperparameters and model
# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# # Print the best hyperparameters
# print("Best Hyperparameters:", best_params)

In [ ]:
# # Retrain the best model on the full training data
# best_logit = LogisticRegression(C=1, penalty='l1', solver='liblinear')
# best_logit.fit(X_train_scaled, y_train)
# # Model Evaluation
# print('Test Accuracy: {:.5f}'.format(best_logit.score(X_test_scaled, y_test)))
# print('Test AUC: {:.5f}'.format(roc_auc_score(y_test, best_logit.predict_proba(X_test_scaled)[:, 1])))

In [ ]:
# # Calculate feature importance
# feature_importance = np.abs(best_logit.coef_[0])
# feature_names = X_train.columns

# # Create a DataFrame to display feature importances
# feature_importance_logit = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
# feature_importance_logit = feature_importance_logit.sort_values(by='Importance', ascending=False)

# # Plot feature importances
# plt.figure(figsize=(10, 6))
# plt.barh(feature_importance_logit['Feature'], feature_importance_logit['Importance'], color='lightgreen')
# plt.xlabel('Importance Score')
# plt.ylabel('Feature')
# plt.title('Feature Importance (Logistic Regression)')
# plt.gca().invert_yaxis()  # Invert y-axis to show most important features at the top
# plt.show()